<a href="https://colab.research.google.com/github/Firenze11/finance_lm/blob/main/llama_for_finance_finetuning.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install transformers datasets peft evaluate huggingface_hub accelerate bitsandbytes

In [1]:
import os
import random
import pandas as pd
import numpy as np
import torch
import transformers

In [2]:
from google.colab import drive

drive.mount('/content/drive')
DATA_DIR = '/content/drive/My Drive/dgm_data/'

BASE_MODEL = "meta-llama/Llama-2-7b-chat-hf"

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [3]:
from huggingface_hub import notebook_login
notebook_login()

## Inferencing

We check the generation results of a baseline Llama2 model.

In [ ]:
from transformers import AutoTokenizer

# tokenizer = AutoTokenizer.from_pretrained(BASE_MODEL)
# pipeline = transformers.pipeline(
#     "text-generation",
#     model=BASE_MODEL,
#     torch_dtype=torch.float16,
#     device_map="auto",
# )

In [ ]:
# sequences = pipeline(
#     """If oil price goes up, what industry sectors will be affected? And which stock ticks are likely a "buy"?""",
#     do_sample=True,
#     top_k=10,
#     num_return_sequences=1,
#     eos_token_id=tokenizer.eos_token_id,
#     max_length=200,
# )
# for seq in sequences:
#     print(f"Result: {seq['generated_text']}")

Result: If oil price goes up, what industry sectors will be affected? And which stock ticks are likely a "buy"?

The oil price is a key driver of economic growth and inflation, and its fluctuations can have far-reaching impacts on various industries. When the oil price increases, it can positively affect industries that are directly or indirectly related to the production, transportation, and consumption of oil and oil products. On the other hand, industries that are negatively affected by higher oil prices may experience decreased demand, increased costs, or both.

Some of the industries that may be positively affected by an increase in oil prices include:

1. Oil and gas exploration and production companies: As oil prices rise, the profitability of these companies increases, leading to higher stock prices.
2. Oilfield services companies: Companies that provide services such as drilling, logging,


## Training

In the following, we compare two versions of fine-tuned Llama2 models.

In [4]:
from datasets import load_dataset, DatasetDict, Dataset

from transformers import (
    AutoTokenizer,
    AutoConfig,
    AutoModelForSequenceClassification,
    AutoModelForCausalLM,
    pipeline,
    DataCollatorWithPadding,
    TrainingArguments,
    Trainer,
    BitsAndBytesConfig)

from peft import PeftModel, PeftConfig, get_peft_model, LoraConfig
import evaluate
tokenizer = AutoTokenizer.from_pretrained(BASE_MODEL)

### Supervised fine-tuning using LoRA

First we fine-tune Llama by giving it news + price_change pairs, and evaluate its prediction performance on price changes.

**Data source:** Daily financial news (mostly analyst ratings) of 6000+ stocks from Kaggle ([Link](https://www.kaggle.com/datasets/miguelaenlle/massive-stock-news-analysis-db-for-nlpbacktests)).

In [ ]:
# define label maps
id2label = {0: "D5+", 1: "D5", 2: "D4", 3: 'D3', 4: 'D2', 5: 'D1', 6: "U1", 7: "U2", 8: "U3", 9: 'U4', 10: 'U5', 11: 'U5+'}
label2id = {val: key for key, val in id2label.items()}
labels = list(id2label.values())

In [ ]:
# generate classification model from model_checkpoint
model = AutoModelForSequenceClassification.from_pretrained(
    BASE_MODEL, num_labels=len(id2label), id2label=id2label, label2id=label2id,
    load_in_8bit=True, device_map='auto')
tokenizer = AutoTokenizer.from_pretrained(BASE_MODEL)

tokenizer.pad_token = tokenizer.eos_token
model.config.pad_token_id = model.config.eos_token_id
data_collator = DataCollatorWithPadding(tokenizer=tokenizer)

config.json:   0%|          | 0.00/614 [00:00<?, ?B/s]

model.safetensors.index.json:   0%|          | 0.00/26.8k [00:00<?, ?B/s]

model-00001-of-00002.safetensors:   0%|          | 0.00/9.98G [00:00<?, ?B/s]

model-00002-of-00002.safetensors:   0%|          | 0.00/3.50G [00:00<?, ?B/s]

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

Some weights of LlamaForSequenceClassification were not initialized from the model checkpoint at meta-llama/Llama-2-7b-chat-hf and are newly initialized: ['score.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


tokenizer_config.json:   0%|          | 0.00/1.62k [00:00<?, ?B/s]

tokenizer.model:   0%|          | 0.00/500k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.84M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/414 [00:00<?, ?B/s]

In [ ]:
# load news data
df = pd.read_csv(DATA_DIR + 'kaggle_financial_news/raw_analyst_ratings.csv')

# generate random labels, TODO: replace this by real stock price change
df['label'] = [random.randint(0, len(labels) - 1) for i in range(len(df))]
df['price_change'] = df['label'].apply(lambda x: id2label[x])
# TODO: replace headline by headline plus full article text
# here we can experiment with different ways of combining the stock symbol and the news content
df['text'] = df.apply(lambda x: f'##stock: {x["stock"]}; ##news: {x["headline"]}', axis=1)
df.head(5)

,Unnamed: 0,headline,url,publisher,date,stock,label,price_change,text
0,0,Stocks That Hit 52-Week Highs On Friday,https://www.benzinga.com/news/20/06/16190091/s...,Benzinga Insights,2020-06-05 10:30:54-04:00,A,1,D5,##stock: A; ##news: Stocks That Hit 52-Week Hi...
1,1,Stocks That Hit 52-Week Highs On Wednesday,https://www.benzinga.com/news/20/06/16170189/s...,Benzinga Insights,2020-06-03 10:45:20-04:00,A,5,D1,##stock: A; ##news: Stocks That Hit 52-Week Hi...
2,2,71 Biggest Movers From Friday,https://www.benzinga.com/news/20/05/16103463/7...,Lisa Levin,2020-05-26 04:30:07-04:00,A,0,D5+,##stock: A; ##news: 71 Biggest Movers From Friday
3,3,46 Stocks Moving In Friday's Mid-Day Session,https://www.benzinga.com/news/20/05/16095921/4...,Lisa Levin,2020-05-22 12:45:06-04:00,A,0,D5+,##stock: A; ##news: 46 Stocks Moving In Friday...
4,4,B of A Securities Maintains Neutral on Agilent...,https://www.benzinga.com/news/20/05/16095304/b...,Vick Meyer,2020-05-22 11:38:59-04:00,A,2,D4,##stock: A; ##news: B of A Securities Maintain...


In [ ]:
df.loc[3, 'url']

'https://www.benzinga.com/news/20/05/16095921/46-stocks-moving-in-fridays-mid-day-session'

In [ ]:
ds = Dataset.from_pandas(df[['text', 'label']][:200])  # TODO: use full dataset
ds = ds.map(lambda samples: tokenizer(samples['text'], truncation=True), batched=True)
ds

Map:   0%|          | 0/200 [00:00<?, ? examples/s]

Asking to truncate to max_length but no maximum length is provided and the model has no predefined maximum length. Default to no truncation.


Dataset({
    features: ['text', 'label', 'input_ids', 'attention_mask'],
    num_rows: 200
})

In [ ]:
# import accuracy evaluation metric
accuracy = evaluate.load("accuracy")

# define an evaluation function to pass into trainer later
def compute_metrics(p):
    predictions, labels = p
    predictions = np.argmax(predictions, axis=1)

    return {"accuracy": accuracy.compute(predictions=predictions,
                                          references=labels)}

In [ ]:
# Untrained model predictions
text_samples = random.sample(df['text'].tolist(), 5)

print("Untrained model predictions:")
print("----------------------------")
for text in text_samples:
    # tokenize text
    inputs = tokenizer.encode(text, return_tensors="pt")
    # compute logits
    logits = model(inputs).logits
    # convert logits to label
    predictions = torch.argmax(logits)

    print(text + " - " + id2label[predictions.tolist()])

Untrained model predictions:
----------------------------
##stock: ANTM; ##news: 92 Biggest Movers From Yesterday - U4
##stock: ABMD; ##news: Abiomed Issues Press Release Highlighting Issuance Of Publication Review Of Observational Analysis Of Impella Previously Presented By Amin At American Heart Association Conference On Nov. 17 - U4
##stock: ADI; ##news: Analog Devices Option Trader Bets $500,000 On Earnings Miss - U5+
##stock: ANTH; ##news: Tonight's Notable Events - U3
##stock: AMID; ##news: Benzinga's M&A Chatter for Tuesday October 14, 2014 - U3


In [ ]:
# Fine-tuning with LoRA
peft_config = LoraConfig(task_type="SEQ_CLS", # sequence classification
                        r=4, # intrinsic rank of trainable weight matrix
                        lora_alpha=32, # this is like a learning rate
                        lora_dropout=0.01, # probablity of dropout
                        target_modules = ['q_proj', 'v_proj']) # we apply lora to query and value layer only

In [ ]:
model = get_peft_model(model, peft_config)
model.print_trainable_parameters()

trainable params: 2,146,304 || all params: 6,609,539,072 || trainable%: 0.03247282415036157


In [ ]:
# hyperparameters
lr = 1e-3 # size of optimization step
batch_size = 16 # number of examples processed per optimziation step
num_epochs = 1 # number of times model runs through training data

# define training arguments
training_args = TrainingArguments(
    output_dir= BASE_MODEL + "-lora-text-classification",
    learning_rate=lr,
    per_device_train_batch_size=batch_size,
    per_device_eval_batch_size=batch_size,
    num_train_epochs=num_epochs,
    weight_decay=0.01,
    evaluation_strategy="epoch",
    save_strategy="epoch",
    load_best_model_at_end=True,
)

In [ ]:
# creater trainer object
trainer = Trainer(
    model=model, # our peft model
    args=training_args, # hyperparameters
    train_dataset=ds, # training data
    eval_dataset=ds, # TODO: using a different validation data
    tokenizer=tokenizer, # define tokenizer
    data_collator=data_collator, # this will dynamically pad examples in each batch to be equal length
    compute_metrics=compute_metrics, # evaluates model using compute_metrics() function from before
)

# train model
trainer.train()

Epoch,Training Loss,Validation Loss,Accuracy
1,No log,nan,{'accuracy': 0.09}


Trainer is attempting to log a value of "{'accuracy': 0.09}" of type <class 'dict'> for key "eval/accuracy" as a scalar. This invocation of Tensorboard's writer.add_scalar() is incorrect so we dropped this attribute.


TrainOutput(global_step=13, training_loss=0.0, metrics={'train_runtime': 12.4485, 'train_samples_per_second': 16.066, 'train_steps_per_second': 1.044, 'total_flos': 605451618680832.0, 'train_loss': 0.0, 'epoch': 1.0})

### Unsupervised fine-tuning using LoRA

Next, we try another method of fine-tuning the model using just the financial analysis articles, without any explicit labels.

**Data source:** articles scraped from the Goldman Sachs market analysis dataset.

#### Train

In [13]:
# model.to('cpu')
del model
import torch
torch.cuda.empty_cache()

In [14]:
bnb_config = BitsAndBytesConfig(
    load_in_4bit=True,
    # bnb_4bit_compute_dtype=torch.bfloat16
)
model = AutoModelForCausalLM.from_pretrained(BASE_MODEL, quantization_config=bnb_config, device_map='auto')  # quantization_config=bnb_config
tokenizer = AutoTokenizer.from_pretrained(BASE_MODEL)
data_collator = DataCollatorWithPadding(tokenizer=tokenizer)

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

In [7]:
# block_size = tokenizer.model_max_length
block_size = 256

def group_texts(examples):
    # Concatenate all texts.
    concatenated_examples = {k: sum(examples[k], []) for k in examples.keys()}
    total_length = len(concatenated_examples[list(examples.keys())[0]])
    # We drop the small remainder, we could add padding if the model supported it instead of this drop, you can customize this part to your needs.
    total_length = (total_length // block_size) * block_size
    # Split by chunks of max_len.
    result = {
        k: [t[i : i + block_size] for i in range(0, total_length, block_size)]
        for k, t in concatenated_examples.items()
    }
    # result['text'] = [tokenizer.decode(l) for l in result["input_ids"]]
    result["labels"] = result["input_ids"].copy()
    return result


# creating a dataset from the scraped text files
scrape_dir = DATA_DIR + 'scraping_gs/'
# data_files = [os.path.join(scrape_dir, fn) for fn in os.listdir(scrape_dir) if fn.startswith('markets')]
# ds2 = load_dataset("text", data_files=['/content/drive/MyDrive/dgm_data/scraping_gs/markets_en_2023_11_15_1490ead5-3921-45db-804c-8c9777fb4f26'])
ds2 = load_dataset('text', data_dir=scrape_dir) #{"train": scrape_dir + 'train/', "eval": scrape_dir + 'eval/'})
ds2 = ds2.map(lambda samples: tokenizer(samples['text'], truncation=True), batched=True, num_proc=4, remove_columns=["text"])
ds2 = ds2.map(group_texts, batched=True, batch_size=1000, num_proc=4)
ds2

Resolving data files:   0%|          | 0/592 [00:00<?, ?it/s]

Resolving data files:   0%|          | 0/264 [00:00<?, ?it/s]

Map (num_proc=4):   0%|          | 0/28743 [00:00<?, ? examples/s]

Map (num_proc=4):   0%|          | 0/10663 [00:00<?, ? examples/s]

DatasetDict({
    train: Dataset({
        features: ['input_ids', 'attention_mask', 'labels'],
        num_rows: 7959
    })
    test: Dataset({
        features: ['input_ids', 'attention_mask', 'labels'],
        num_rows: 3636
    })
})

In [16]:
# Fine-tuning with LoRA
peft_config = LoraConfig(task_type="CAUSAL_LM", # next token prediction
                        r=4, # intrinsic rank of trainable weight matrix
                        lora_alpha=32, # this is like a learning rate
                        lora_dropout=0.05, # probablity of dropout
                        target_modules = ['q_proj', 'v_proj']) # we apply lora to query and value layer only

In [17]:
model2train = get_peft_model(model, peft_config)
model2train.print_trainable_parameters()

AttributeError: ignored

In [9]:
# hyperparameters
lr = 2e-5 # size of optimization step
batch_size = 8 # number of examples processed per optimziation step
num_epochs = 10 # number of times model runs through training data

# define training arguments
training_args = TrainingArguments(
    output_dir= DATA_DIR + "models/lora-causal-lm",
    learning_rate=lr,
    per_device_train_batch_size=batch_size,
    num_train_epochs=num_epochs,
    weight_decay=0.01,
    save_strategy="epoch"
)

In [ ]:
task_evaluator = evaluate.evaluator("text-generation")

# 1. Pass a model name or path
eval_results = task_evaluator.compute(
    model_or_pipeline=pipeline(task='text-generation', model=model2train, tokenizer=tokenizer),
    data=ds2['test'].select(range(2)),
    input_column='text',
    label_column='text'
)
eval_results

KeyboardInterrupt: ignored

In [25]:
# creater trainer object
trainer = Trainer(
    model=model2train, # our peft model
    args=training_args, # hyperparameters
    train_dataset=ds2['train'], # training data
    eval_dataset=ds2['test'], # eval data
)

# evaluate untrained model
trainer.evaluate()

{'eval_loss': 3.0671470165252686,
 'eval_runtime': 1228.2662,
 'eval_samples_per_second': 2.96,
 'eval_steps_per_second': 0.37}

In [ ]:
# creater trainer object
trainer = Trainer(
    model=model2train, # our peft model
    args=training_args, # hyperparameters
    train_dataset=ds2['train'], # training data
    eval_dataset=ds2['test'], # eval data
)

# evaluate untrained model
trainer.evaluate()

In [15]:
# post-hoc evaluate trained model
adapter_model_name = '/content/drive/MyDrive/dgm_data/models/lora-causal-lm/checkpoint-995'
model_peft = PeftModel.from_pretrained(model, adapter_model_name)

trainer2 = Trainer(
    model=model_peft, # our peft model
    args=training_args, # hyperparameters
    train_dataset=ds2['train'], # training data
    eval_dataset=ds2['test'], # eval data
)

# evaluate untrained model
trainer2.evaluate()

/usr/local/lib/python3.10/dist-packages/bitsandbytes/nn/modules.py:228: UserWarning: Input type into Linear4bit is torch.float16, but bnb_4bit_compute_type=torch.float32 (default). This will lead to slow inference or training speed.
  warnings.warn(f'Input type into Linear4bit is torch.float16, but bnb_4bit_compute_type=torch.float32 (default). This will lead to slow inference or training speed.')


{'eval_loss': 2.3975412845611572,
 'eval_runtime': 959.4528,
 'eval_samples_per_second': 3.79,
 'eval_steps_per_second': 0.474}

In [ ]:
# train model
trainer.train()

/usr/local/lib/python3.10/dist-packages/bitsandbytes/nn/modules.py:226: UserWarning: Input type into Linear4bit is torch.float16, but bnb_4bit_compute_type=torch.float32 (default). This will lead to slow inference or training speed.
  warnings.warn(f'Input type into Linear4bit is torch.float16, but bnb_4bit_compute_type=torch.float32 (default). This will lead to slow inference or training speed.')


Step,Training Loss
500,2.475900
1000,2.292500


KeyboardInterrupt: ignored

In [ ]:
import math
eval_results = trainer.evaluate()
print(f"Perplexity: {math.exp(eval_results['eval_loss']):.2f}")

Step,Training Loss
500,2.475900
1000,2.292500


KeyboardInterrupt: ignored

#### Inference

In [ ]:
bnb_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_use_double_quant=True,
    bnb_4bit_quant_type="nf4",
    bnb_4bit_compute_dtype=torch.bfloat16
)

In [ ]:
import gc
model.cpu()
del model
gc.collect()
torch.cuda.empty_cache()

In [ ]:
model = AutoModelForCausalLM.from_pretrained(BASE_MODEL, device_map='auto')
adapter_model_name = '/content/drive/MyDrive/dgm_data/models/lora-causal-lm/checkpoint-995'
model_peft = PeftModel.from_pretrained(model, adapter_model_name)

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

In [ ]:
model_peft.push_to_hub('llama2-lora-finance')

NotImplementedError: ignored

In [ ]:
device = "cuda"
model_peft = model_peft.merge_and_unload()
inputs = tokenizer.encode("If oil price goes up, what industry sectors will be affected? And which stock ticks are likely a 'buy'?", return_tensors="pt").to(device)
outputs = model_peft.generate(inputs)
print(tokenizer.decode(outputs[0]))

/usr/local/lib/python3.10/dist-packages/bitsandbytes/nn/modules.py:226: UserWarning: Input type into Linear4bit is torch.float16, but bnb_4bit_compute_type=torch.float32 (default). This will lead to slow inference or training speed.
  warnings.warn(f'Input type into Linear4bit is torch.float16, but bnb_4bit_compute_type=torch.float32 (default). This will lead to slow inference or training speed.')


KeyboardInterrupt: ignored

In [ ]:
device = "cuda"
model_peft.to(device)
model_peft = model_peft.merge_and_unload()
inputs = tokenizer.encode("If oil price goes up, what industry sectors will be affected? And which stock ticks are likely a 'buy'?", return_tensors="pt").to(device)
outputs = model_peft.generate(inputs)
print(tokenizer.decode(outputs[0]))

<s> If oil price goes up, what industry sectors will be affected? And which stock ticks are likely a 'buy'?

The oil price is closely linked to the overall economy, and when oil prices rise, it can have a broad impact on the economy. Here are some of the industry sectors that could be impacted by higher oil prices:

1. Energy and Utilities: The energy and utilities sector is the most obvious sector that will be impacted by higher oil prices. Companies in this sector include ExxonMobil (XOM), Chevron (CVX), ConocoPhillips (COP), and Valero Energy (VLO).
2. Transportation: Companies in the transportation sector, such as airlines and trucking companies, could see increased fuel costs. Companies in this sector include American Airlines Group (AAL), United Airlines Holdings (UAL), Southwest Airlines Co. (LUV), and J.B. Hunt Transport Services (JBHT).
3. Consumer Discretionary: Companies in the consumer discretionary sector, such as automakers, could see increased costs for raw materials and

In [ ]:
model.to(device)
outputs = model.generate(inputs)
print(tokenizer.decode(outputs[0]))

<s> If oil price goes up, what industry sectors will be affected? And which stock ticks are likely a 'buy'? <s>    1) Oil price and its impact on the economy and the stock market    2) Sector-by-sector impact of oil price increase    3) Which stocks are likely to benefit from an increase in oil price<s>        4) Oil price outlook and potential impact on the stock market<s>        5) Conclusion<s><s> The oil price has been on a downward trend since 2014 and the OPEC (Organization of the Petroleum Exporting Countries) has been instrumental in this trend. The oil price has been influenced by various factors, including the rise of renewable energy, the slowdown in global economic growth, and the increase in supply from the US shale oil industry. The OPEC has been actively managing the oil price through production cuts and has been successful in reducing the global supply of oil. However, the oil price has been fluctuating due to various factors, including the ongoing geopolitical tensions

In [ ]:
device = "cuda"
model.to(device)
model = model.merge_and_unload()
inputs = tokenizer.encode("If oil price goes up, what industry sectors will be affected? And which stock ticks are likely a 'buy'?", return_tensors="pt").to(device)
outputs = model.generate(inputs)
print(tokenizer.decode(outputs[0]))

<s> If oil price goes up, what industry sectors will be affected? And which stock ticks are likely a 'buy'?<s>
Oil prices have been on a rollercoaster ride this year, with prices surging to a 14-year high of $127/bbl in March on the back of the Russia-Ukraine conflict, before plunging to a 7-month low of $70/bbl in July. As of 26 Oct, Brent crude oil prices are trading at $86/bbl, up 22% from the beginning of the year. The upside in oil prices has led to a rally in the energy sector, with the Energy Select Sector SPDR ETF (XLE) up 24% year-to-date. In this article, we will discuss the impact of rising oil prices on various industry sectors and identify potential 'buy' stocks. <s>

Impact of rising oil prices on industry sectors:

1. Oil & Gas Exploration & Production (E&P) companies: Oil & gas E&P companies are directly impacted by rising oil prices, as higher prices increase their profitability and cash flow. The sector has seen a significant rally this year, with the XLE up 30%. The 

#### Evaluate

In [ ]:
eval_scenarios = [
    {
        'category': 'Macro level - Economy, Market, Geopolitics',
        'description': 'Understand the market, predict market trends and sector allocation opportunities, and list related ETFs',
        'text': 'With inflation remaining high yet more controlled, what are the prevailing trends in the current market? Analyze the anticipated performance of the U.S. equity market, identifying which sectors are likely to benefit and which may face challenges. Additionally, please provide a list of relevant ETFs corresponding to the sectors discussed.'
    },
    {
        'category': 'Geopolitical Impact and Long / Short Opportunities',
        'description': '',
        'text': 'What are the predominant geopolitical risks at present, particularly concerning the relationship between the U.S. and China, and their potential impact on equity markets? Focus on the technology sector, especially semiconductors, analyzing the effects from policy decisions, sector dynamics, and company revenue angles. Could you provide a list of tech companies that are negatively and positively influenced by these geopolitical tensions? Please include their stock tickers.'
    },
    {
        'category': 'Micro level - Company',
        'description': 'Research the company - understand a new product’s potential',
        'text': "Apple's recent launch of the Vision Pro has significant implications for the company's fundamentals. Analyze its impact, focusing on direct revenue generation, margin expansion, and the strengthening of Apple's ecosystem. How does the Vision Pro compare to the iPhone or iPad in terms of influence? Additionally, provide a brief history of how the iPhone and iPad have shaped Apple's development and success."
    }
]

In [ ]:
for i, es in enumerate(eval_scenarios):
    print(f'---------------- Scenario {i} ---------------')
    device = "cuda"
    inputs = tokenizer.encode(es['text'], return_tensors="pt").to(device)
    outputs = model.generate(inputs)
    print(tokenizer.decode(outputs[0]))

---------------- Scenario 0 ---------------


/usr/local/lib/python3.10/dist-packages/bitsandbytes/nn/modules.py:228: UserWarning: Input type into Linear4bit is torch.float16, but bnb_4bit_compute_type=torch.float32 (default). This will lead to slow inference or training speed.
  warnings.warn(f'Input type into Linear4bit is torch.float16, but bnb_4bit_compute_type=torch.float32 (default). This will lead to slow inference or training speed.')


<s> With inflation remaining high yet more controlled, what are the prevailing trends in the current market? Analyze the anticipated performance of the U.S. equity market, identifying which sectors are likely to benefit and which may face challenges. Additionally, please provide a list of relevant ETFs corresponding to the sectors discussed.<s> What are the prevailing trends in the current market?<s> With inflation remaining high yet more controlled, what are the prevailing trends in the current market? Analyze the anticipated performance of the U.S. equity market, identifying which sectors are likely to benefit and which may face challenges. Additionally, please provide a list of relevant ETFs corresponding to the sectors discussed.<s> <s>         Market Overview<s>  The U.S. equity market has been on a strong run since the start of the year, driven by a combination of strong earnings growth, a more favorable economic backdrop, and a less accommodative Federal Reserve. The S&P 500 Ind

In [ ]:
device = "cuda"
model_peft.to(device)
model_peft = model_peft.merge_and_unload()

for i, es in enumerate(eval_scenarios[-1:]):
    print(f'---------------- Scenario {i} ---------------')
    inputs = tokenizer.encode(es['text'], return_tensors="pt").to(device)
    outputs = model_peft.generate(inputs)
    print(tokenizer.decode(outputs[0]))

---------------- Scenario 0 ---------------
<s> With inflation remaining high yet more controlled, what are the prevailing trends in the current market? Analyze the anticipated performance of the U.S. equity market, identifying which sectors are likely to benefit and which may face challenges. Additionally, please provide a list of relevant ETFs corresponding to the sectors discussed.<s>     Key Themes and ETFs:<s> 1. Sector Outlook: Higher for Longer: The market is likely to continue to benefit from the macroeconomic backdrop of high growth, high inflation, and high interest rates, which should support the overall equity market. However, we expect the market to become more selective as earnings growth slows down, and the market becomes more focused on the companies that can deliver earnings growth. In the short term, we expect the technology sector to outperform as the earnings growth in this sector is likely to be higher than the overall market. In the long term, we expect the sector

In [ ]:
for i, es in enumerate(eval_scenarios[-1:]):
    print(f'---------------- Scenario {i} ---------------')
    inputs = tokenizer.encode(es['text'], return_tensors="pt").to(device)
    outputs = model_peft.generate(inputs)
    print(tokenizer.decode(outputs[0]))

---------------- Scenario 0 ---------------
<s> Apple's recent launch of the Vision Pro has significant implications for the company's fundamentals. Analyze its impact, focusing on direct revenue generation, margin expansion, and the strengthening of Apple's ecosystem. How does the Vision Pro compare to the iPhone or iPad in terms of influence? Additionally, provide a brief history of how the iPhone and iPad have shaped Apple's development and success. <s> Exhibit 1: Apple's revenue breakdown for FY2022, with AR/VR/Mixed Reality at 0.3% of revenue<s> Exhibit 2: Apple's revenue breakdown for FY2022, with AR/VR/Mixed Reality at 0.3% of revenue<s> Exhibit 3: Apple's revenue breakdown for FY2022, with AR/VR/Mixed Reality at 0.3% of revenue<s> Exhibit 4: Apple's revenue breakdown for FY2022, with AR/VR/Mixed Reality at 0.3% of revenue<s> In terms of direct revenue generation, the Vision Pro has the potential to contribute to Apple's top line, given its targeted market of enterprises, educat